<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/VoiceCloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ### Clonación de Voz en Google Colab

# **Nota:** Asegúrate de ejecutar este bloque en una instancia de GPU en Google Colab para acelerar el proceso.
#
# 1. Ve a **Entorno de ejecución > Cambiar tipo de entorno de ejecución**.
# 2. En **Acelerador de hardware**, selecciona **GPU** y guarda.

# ```python
# Paso 1: Instalar Dependencias Básicas
!apt-get update
!apt-get install -y git ffmpeg libsndfile1

# Paso 2: Clonar el Repositorio de Real-Time Voice Cloning
!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git
%cd Real-Time-Voice-Cloning/

# Paso 3: Instalar Dependencias de Python
!pip install -q -r requirements.txt
!pip install -q tensorflow==1.15
!pip install -q keras==2.3.1
!pip install -q PyQt5==5.12
!pip install -q gdown
!pip install -q soundfile

# Paso 4: Descargar Modelos Preentrenados
# Descargar modelos utilizando gdown
import os

# Crear carpeta para modelos preentrenados si no existe
if not os.path.exists("pretrained_models"):
    os.makedirs("pretrained_models")

# IDs de Google Drive para los modelos preentrenados
model_ids = {
    "encoder": "1Q8zwO-TC3GJ7IrmOvV8XG6NJsWpCzYfj",       # Reemplaza con el ID correcto
    "synthesizer": "1uVZ7zZ7i7VgM9bVcVZpjCq2E1xJt4JbX",   # Reemplaza con el ID correcto
    "vocoder": "1RZxG6xEwJZbV9S9K1XK1YxY2xE3xF4G5"        # Reemplaza con el ID correcto
}

# Función para descargar modelos
def download_model(model_name, id):
    destination = f"pretrained_models/{model_name}.zip"
    if not os.path.exists(destination):
        !gdown --id {id} -O {destination}
        !unzip -o {destination} -d pretrained_models/
    else:
        print(f"El modelo {model_name} ya está descargado.")

# Descargar cada modelo
for model_name, model_id in model_ids.items():
    download_model(model_name, model_id)

# Paso 5: Subir Archivos de Audio
from google.colab import files
uploaded = files.upload()

# Paso 6: Clonación de Voz
import numpy as np
import librosa
import IPython.display as ipd
import soundfile as sf
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder

# Cargar el sintetizador
synthesizer = Synthesizer("pretrained_models/synthesizer/synthesizer.pt")

# Cargar el codificador
encoder.load_model("pretrained_models/encoder/saved_models/pretrained.pt")

# Cargar el vocoder
vocoder.load_model("pretrained_models/vocoder/saved_models/pretrained.pt")

# Función para clonar la voz
def clone_voice(audio_path, text, output_path="audio_generado.wav"):
    # Preprocesar el audio de entrada
    encoder_wav = encoder.preprocess_wav(audio_path)
    encoder_embedding = encoder.embed_utterance(encoder_wav)

    # Sintetizar el espectrograma mel
    specs = synthesizer.synthesize_spectrograms([text], [encoder_embedding])

    # Generar el audio a partir del espectrograma mel
    generated_wav = vocoder.infer_waveform(specs[0])

    # Normalizar el audio generado
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    generated_wav = librosa.util.normalize(generated_wav)

    # Guardar el audio generado
    sf.write(output_path, generated_wav, synthesizer.sample_rate)

    return output_path

# Obtener el nombre del archivo de audio subido
if len(uploaded) == 0:
    raise ValueError("No se ha subido ningún archivo de audio. Por favor, sube un archivo .wav.")
audio_file = list(uploaded.keys())[0]

# Definir el texto que deseas sintetizar
texto = "Este es un ejemplo de clonación de voz utilizando muestras de audio proporcionadas."

# Clonar la voz
print("Generando audio...")
audio_generado_path = clone_voice(audio_file, texto)
print("Audio generado exitosamente.")

# Reproducir el audio generado
ipd.display(ipd.Audio(audio_generado_path))

# Paso 7: Descargar el Audio Generado (Opcional)
files.download(audio_generado_path)